In [1]:
import pandas as pd

import sys
sys.path.append('../utils')
import classifier_utils

# Load Data

In [2]:
data_path = "../../data/fine-tuning/"

In [3]:
df = pd.concat([
    pd.read_csv(data_path + "moravian_deu_train.csv", index_col=0),
    pd.read_csv(data_path + "moravian_deu_test.csv", index_col=0) 
])
df

,text,label
source,,
11,Ich küßte Ihm im Geist die Hand mit ihren Näge...,positive
37,Ihm warst du ganz geweiht!,positive
19,Ein Soldaten Bruder der bey meinem Meister im ...,positive
34,So weit ihr eigener Aufsatz.,neutral
21,da er noch ein paar Jahr als Gärtner u. nun 23...,negative
...,...,...
9,Die lezte Zeit kränckelte er.,negative
38,Ich sahe sie oft auf ihrem Angesicht liegen un...,positive
13,Ich bat den Heiland mein Herz mit seinem Blute...,positive


## Add Index and Polarity Score

In [4]:
df = df.reset_index()

df["polarity"] = df["label"].apply(lambda x: classifier_utils.get_polarity(x))
df

,source,text,label,polarity
0,11,Ich küßte Ihm im Geist die Hand mit ihren Näge...,positive,1
1,37,Ihm warst du ganz geweiht!,positive,1
2,19,Ein Soldaten Bruder der bey meinem Meister im ...,positive,1
3,34,So weit ihr eigener Aufsatz.,neutral,0
4,21,da er noch ein paar Jahr als Gärtner u. nun 23...,negative,-1
...,...,...,...,...
2205,9,Die lezte Zeit kränckelte er.,negative,-1
2206,38,Ich sahe sie oft auf ihrem Angesicht liegen un...,positive,1
2207,13,Ich bat den Heiland mein Herz mit seinem Blute...,positive,1
2208,34,nie hört’ man dich klagen;,positive,1


# Find Instances with "Blood and Wounds" Tokens

In [5]:
sample_df = df.loc[(df["text"].str.contains("Blut")) | (df["text"].str.contains("Wunden")) | (df["text"].str.contains("blut"))]
sample_df

,source,text,label,polarity
42,8,"Im Merz 58 zog er nach Naz. Hall, die Orgel da...",positive,1
73,5,So gewohnte ich nach und nach in diese neue Ar...,positive,1
115,34,"Ja! geliebte Magd des HErrn, die von uns gesch...",positive,1
133,34,Durch mein Bleiben zog ich meiner Tante großen...,negative,-1
141,9,Ich las einmal in {NAME}s wahrem Christenthum ...,negative,-1
...,...,...,...,...
2085,12,"Allemal, wenn in der Stube geräuchert wurde, w...",negative,-1
2110,13,Unser nächstes Chorfest am 4ten May sowol als ...,positive,1
2168,3,"Der Vers: ""Erschein mir in dem Bilde, wie Du f...",positive,1
2171,5,Es war aber der Periodus der Wunden- Litaney u...,positive,1


# Get Model Predictions

In [6]:
prediction_path = "../../data/predictions/"

## Off-the-shelf Lexicon-based Approaches

### SentiWS

In [7]:
sentiws_df = pd.concat([
    pd.read_csv(prediction_path + "moravian_deu_train_SentiWS.csv", index_col=0),
    pd.read_csv(prediction_path + "moravian_deu_test_SentiWS.csv", index_col=0) 
]).reset_index()
sentiws_df

,source,text,polarity
0,11,Ich küßte Ihm im Geist die Hand mit ihren Näge...,0.000000
1,37,Ihm warst du ganz geweiht!,0.000000
2,19,Ein Soldaten Bruder der bey meinem Meister im ...,0.014444
3,34,So weit ihr eigener Aufsatz.,0.000000
4,21,da er noch ein paar Jahr als Gärtner u. nun 23...,0.032749
...,...,...,...
2205,9,Die lezte Zeit kränckelte er.,0.000000
2206,38,Ich sahe sie oft auf ihrem Angesicht liegen un...,0.000000
2207,13,Ich bat den Heiland mein Herz mit seinem Blute...,0.004317
2208,34,nie hört’ man dich klagen;,0.000000


### GerVADER

In [8]:
gervader_df = pd.concat([
    pd.read_csv(prediction_path + "moravian_deu_train_GerVADER.csv", index_col=0),
    pd.read_csv(prediction_path + "moravian_deu_test_GerVADER.csv", index_col=0) 
]).reset_index()
gervader_df

,source,text,polarity
0,11,Ich küßte Ihm im Geist die Hand mit ihren Näge...,0.4939
1,37,Ihm warst du ganz geweiht!,0.0000
2,19,Ein Soldaten Bruder der bey meinem Meister im ...,0.6124
3,34,So weit ihr eigener Aufsatz.,0.0000
4,21,da er noch ein paar Jahr als Gärtner u. nun 23...,0.8845
...,...,...,...
2205,9,Die lezte Zeit kränckelte er.,0.0000
2206,38,Ich sahe sie oft auf ihrem Angesicht liegen un...,0.0000
2207,13,Ich bat den Heiland mein Herz mit seinem Blute...,0.5719
2208,34,nie hört’ man dich klagen;,-0.2500


## Off-the-shelf Transformer Models

### ger-senti-bert

In [9]:
ger_senti_bert_df = pd.concat([
    pd.read_csv(prediction_path + "moravian_deu_train_ger-senti-bert.csv", index_col=0),
    pd.read_csv(prediction_path + "moravian_deu_test_ger-senti-bert.csv", index_col=0) 
]).reset_index()
ger_senti_bert_df["polarity"] = ger_senti_bert_df["label"].apply(lambda x: classifier_utils.get_polarity(x))
ger_senti_bert_df

,source,text,label,polarity
0,11,Ich küßte Ihm im Geist die Hand mit ihren Näge...,neutral,0
1,37,Ihm warst du ganz geweiht!,neutral,0
2,19,Ein Soldaten Bruder der bey meinem Meister im ...,neutral,0
3,34,So weit ihr eigener Aufsatz.,neutral,0
4,21,da er noch ein paar Jahr als Gärtner u. nun 23...,neutral,0
...,...,...,...,...
2205,9,Die lezte Zeit kränckelte er.,neutral,0
2206,38,Ich sahe sie oft auf ihrem Angesicht liegen un...,neutral,0
2207,13,Ich bat den Heiland mein Herz mit seinem Blute...,neutral,0
2208,34,nie hört’ man dich klagen;,positive,1


### senti-distilbert

In [10]:
senti_distilbert_df = pd.concat([
    pd.read_csv(prediction_path + "moravian_deu_train_senti-distilbert.csv", index_col=0),
    pd.read_csv(prediction_path + "moravian_deu_test_senti-distilbert.csv", index_col=0) 
]).reset_index()
senti_distilbert_df["polarity"] = senti_distilbert_df["label"].apply(lambda x: classifier_utils.get_polarity(x))
senti_distilbert_df

,source,text,label,polarity
0,11,Ich küßte Ihm im Geist die Hand mit ihren Näge...,positive,1
1,37,Ihm warst du ganz geweiht!,positive,1
2,19,Ein Soldaten Bruder der bey meinem Meister im ...,negative,-1
3,34,So weit ihr eigener Aufsatz.,positive,1
4,21,da er noch ein paar Jahr als Gärtner u. nun 23...,negative,-1
...,...,...,...,...
2205,9,Die lezte Zeit kränckelte er.,negative,-1
2206,38,Ich sahe sie oft auf ihrem Angesicht liegen un...,negative,-1
2207,13,Ich bat den Heiland mein Herz mit seinem Blute...,negative,-1
2208,34,nie hört’ man dich klagen;,negative,-1


## Fine-tuned Models

### bert*

In [11]:
finetuned_bert_path = "../../data/predictions/moravian_deu_test_bert_fine-tuned.csv"
finetuned_bert_df = pd.concat([
    pd.read_csv(prediction_path + "moravian_deu_train_bert_fine-tuned.csv", index_col=0),
    pd.read_csv(prediction_path + "moravian_deu_test_bert_fine-tuned.csv", index_col=0) 
]).reset_index()
finetuned_bert_df["polarity"] = finetuned_bert_df["label"].apply(lambda x: classifier_utils.get_polarity(x))
finetuned_bert_df

,source,text,label,polarity
0,11,Ich küßte Ihm im Geist die Hand mit ihren Näge...,positive,1
1,37,Ihm warst du ganz geweiht!,positive,1
2,19,Ein Soldaten Bruder der bey meinem Meister im ...,positive,1
3,34,So weit ihr eigener Aufsatz.,neutral,0
4,21,da er noch ein paar Jahr als Gärtner u. nun 23...,negative,-1
...,...,...,...,...
2205,9,Die lezte Zeit kränckelte er.,positive,1
2206,38,Ich sahe sie oft auf ihrem Angesicht liegen un...,positive,1
2207,13,Ich bat den Heiland mein Herz mit seinem Blute...,positive,1
2208,34,nie hört’ man dich klagen;,positive,1


### gbert*

In [12]:
finetuned_gbert_df = pd.concat([
    pd.read_csv(prediction_path + "moravian_deu_train_gbert_fine-tuned.csv", index_col=0),
    pd.read_csv(prediction_path + "moravian_deu_test_gbert_fine-tuned.csv", index_col=0) 
]).reset_index()
finetuned_gbert_df["polarity"] = finetuned_gbert_df["label"].apply(lambda x: classifier_utils.get_polarity(x))
finetuned_gbert_df

,source,text,label,polarity
0,11,Ich küßte Ihm im Geist die Hand mit ihren Näge...,positive,1
1,37,Ihm warst du ganz geweiht!,positive,1
2,19,Ein Soldaten Bruder der bey meinem Meister im ...,positive,1
3,34,So weit ihr eigener Aufsatz.,neutral,0
4,21,da er noch ein paar Jahr als Gärtner u. nun 23...,negative,-1
...,...,...,...,...
2205,9,Die lezte Zeit kränckelte er.,negative,-1
2206,38,Ich sahe sie oft auf ihrem Angesicht liegen un...,positive,1
2207,13,Ich bat den Heiland mein Herz mit seinem Blute...,positive,1
2208,34,nie hört’ man dich klagen;,positive,1


### ger-senti-bert*

In [13]:
finetuned_ger_senti_bert_path = "../../data/predictions/moravian_deu_test_ger_senti_bert_fine-tuned.csv"
finetuned_ger_senti_bert_df = pd.concat([
    pd.read_csv(prediction_path + "moravian_deu_train_ger-senti-bert_fine-tuned.csv", index_col=0),
    pd.read_csv(prediction_path + "moravian_deu_test_ger-senti-bert_fine-tuned.csv", index_col=0) 
]).reset_index()
finetuned_ger_senti_bert_df["polarity"] = finetuned_ger_senti_bert_df["label"].apply(lambda x: classifier_utils.get_polarity(x))
finetuned_ger_senti_bert_df

,source,text,label,polarity
0,11,Ich küßte Ihm im Geist die Hand mit ihren Näge...,positive,1
1,37,Ihm warst du ganz geweiht!,positive,1
2,19,Ein Soldaten Bruder der bey meinem Meister im ...,positive,1
3,34,So weit ihr eigener Aufsatz.,neutral,0
4,21,da er noch ein paar Jahr als Gärtner u. nun 23...,negative,-1
...,...,...,...,...
2205,9,Die lezte Zeit kränckelte er.,negative,-1
2206,38,Ich sahe sie oft auf ihrem Angesicht liegen un...,positive,1
2207,13,Ich bat den Heiland mein Herz mit seinem Blute...,positive,1
2208,34,nie hört’ man dich klagen;,positive,1


# Compile Overview

In [14]:
data = {
    "manual": [df["polarity"].mean(), sample_df["polarity"].mean()],
    "SentiWS": [sentiws_df["polarity"].mean(), sentiws_df.iloc[sample_df.index]["polarity"].mean()],
    "GerVADER": [gervader_df["polarity"].mean(), gervader_df.iloc[sample_df.index]["polarity"].mean()],
    "ger-senti-bert": [ger_senti_bert_df["polarity"].mean(), ger_senti_bert_df.iloc[sample_df.index]["polarity"].mean()],
    "senti-distilbert": [senti_distilbert_df["polarity"].mean(), senti_distilbert_df.iloc[sample_df.index]["polarity"].mean()],
    "bert*": [finetuned_bert_df["polarity"].mean(), finetuned_bert_df.iloc[sample_df.index]["polarity"].mean()],
    "gbert*": [finetuned_gbert_df["polarity"].mean(), finetuned_gbert_df.iloc[sample_df.index]["polarity"].mean()],
    "ger-senti-bert*": [finetuned_ger_senti_bert_df["polarity"].mean(), finetuned_ger_senti_bert_df.iloc[sample_df.index]["polarity"].mean()],
}

pd.DataFrame.from_dict(data, orient="index", columns=["Mean_Sentiment", "Mean_Sentiment_Sample"])

,Mean_Sentiment,Mean_Sentiment_Sample
manual,0.152489,0.517241
SentiWS,0.001936,-0.000344
GerVADER,0.176953,0.199802
ger-senti-bert,-0.042986,-0.011494
senti-distilbert,-0.055204,0.080460
bert*,0.142534,0.574713
gbert*,0.161538,0.517241
ger-senti-bert*,0.166516,0.551724
